In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

**Step 2: Load Dataset Paths**

Set the paths for training and testing datasets.

In [ ]:
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'

# Helper function to load file paths and labels
def load_data(directory):
    filepaths = []
    labels = []
    for fold in os.listdir(directory):
        foldpath = os.path.join(directory, fold)
        for f in os.listdir(foldpath):
            filepaths.append(os.path.join(foldpath, f))
            labels.append(fold)
    return pd.DataFrame(data={'filepaths': filepaths, 'labels': labels})

train_df = load_data(train_dir)
ts_df = load_data(test_dir)

# Split test data for validation and test sets
valid_df, test_df = train_test_split(ts_df, test_size=0.5, random_state=42)


**Step 3: Data Augmentation and Generators**

Prepare the data for training using augmentation techniques.

In [ ]:
# Data Augmentation for training data
train_datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Only rescale for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create Generators
train_gen = train_datagen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=(224, 224),
                                              color_mode='rgb', class_mode='categorical', batch_size=32)

valid_gen = val_test_datagen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=(224, 224),
                                                 color_mode='rgb', class_mode='categorical', batch_size=32)

test_gen = val_test_datagen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=(224, 224),
                                                color_mode='rgb', class_mode='categorical', batch_size=32)


**Step 4: Model Architecture**

Define the architecture of the convolutional neural network (CNN).

In [ ]:
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu'),
    Dropout(0.4),
    
    Dense(4, activation='softmax')  # Output layer for 4 classes
])

model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


**Step 5: Model Training with Early Stopping and Model Checkpoint**

Train the model using the training data, with callbacks to stop training early if the validation loss does not improve.

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('brain_tumor_best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(train_gen, validation_data=valid_gen, epochs=30, callbacks=[early_stopping, model_checkpoint])


**Step 6: Training History Visualization**

Visualize the training and validation loss and accuracy.

In [ ]:
# Plot Training History
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


**Step 7: Model Evaluation**

Evaluate the model's performance on the training, validation, and test datasets.

In [ ]:
# Evaluate the model on each dataset
train_loss, train_acc = model.evaluate(train_gen, verbose=0)
valid_loss, valid_acc = model.evaluate(valid_gen, verbose=0)
test_loss, test_acc = model.evaluate(test_gen, verbose=0)

# Plot Loss and Accuracy Comparison
categories = ['Train', 'Validation', 'Test']
loss_values = [train_loss, valid_loss, test_loss]
accuracy_values = [train_acc, valid_acc, test_acc]

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].bar(categories, loss_values, color=['blue', 'orange', 'green'])
ax[0].set_title('Loss Comparison')
ax[0].set_xlabel('Dataset')
ax[0].set_ylabel('Loss')

ax[1].bar(categories, accuracy_values, color=['blue', 'orange', 'green'])
ax[1].set_title('Accuracy Comparison')
ax[1].set_xlabel('Dataset')
ax[1].set_ylabel('Accuracy')
plt.show()


**Step 8: Prediction on Test Images and Model Saving**

Load the best model, generate predictions, and save the trained model.

In [ ]:
# Load the best model and evaluate on test data
best_model = model.load_weights('brain_tumor_best_model.h5')

# Generate predictions
predictions = np.argmax(best_model.predict(test_gen), axis=1)
true_labels = test_gen.classes

# Classification report
print(classification_report(true_labels, predictions, target_names=test_gen.class_indices.keys()))

# Save the trained model
model.save('final_brain_tumor_classification_model.h5')


**Step 9: Predict Disease on New Image**

Create a function to predict the class of a new image.

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict the class
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_labels = list(test_gen.class_indices.keys())
    return class_labels[class_index], prediction[0][class_index]

# Test the function with an image from the test directory
sample_image = test_df.sample(1).filepaths.values[0]
predicted_class, confidence = predict_image(sample_image, model)
print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")
